In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import pandas_ta as ta

from gplearn.extra_functions_2 import *

from numpy.lib.stride_tricks import sliding_window_view

In [3]:
data = np.random.rand(4000, 300) 
data[::50] = np.nan
data

array([[       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [0.40403407, 0.92446624, 0.55997877, ..., 0.85203431, 0.75658877,
        0.17587631],
       [0.22497359, 0.82349439, 0.57884572, ..., 0.06413619, 0.97799023,
        0.14544993],
       ...,
       [0.52952234, 0.37550779, 0.90555013, ..., 0.71766411, 0.98577732,
        0.1006726 ],
       [0.70113341, 0.91370592, 0.277089  , ..., 0.50396947, 0.94832041,
        0.62943199],
       [0.7933592 , 0.30038741, 0.79236493, ..., 0.02243592, 0.83097011,
        0.3548127 ]])

In [10]:
dffct = pd.read_parquet('/home/pennymax/proj/MFM/CrossSection/v2_8h/prepare_data/fct_proc.parquet')
dffct

Open       High        Low  \
time                      symbol                                      
2020-06-21 00:00:00+00:00 ADAUSDT     0.07941    0.08099    0.07924   
                          ATOMUSDT    2.62500    2.63100    2.59900   
                          BATUSDT     0.23050    0.23530    0.22970   
                          BCHUSDT   232.69000  234.49000  232.34000   
                          BNBUSDT    16.07600   16.33500   16.03500   
...                                       ...        ...        ...   
2024-02-10 16:00:00+00:00 YGGUSDT     0.44940    0.45510    0.44910   
                          ZECUSDT    20.93000   21.09000   20.83000   
                          ZENUSDT     7.94900    8.07300    7.93300   
                          ZILUSDT     0.02084    0.02118    0.02079   
                          ZRXUSDT     0.31920    0.32190    0.31840   

                                        Close        Volume        QuoVol  \
time                      symbol                                            
2020-06-21 00:00:00+00:00 ADAUSDT     0.07988  1.289528e+08  1.030266e+07   
                          ATOMUSDT    2.60000  2.186062e+05  5.718994e+05   
                          BATUSDT     0.23080  1.195930e+07  2.778131e+06   
                          BCHUSDT   233.27000  3.490217e+04  8.138722e+06   
                          BNBUSDT    16.04600  2.257289e+05  3.649726e+06   
...                                       ...           ...           ...   
2024-02-10 16:00:00+00:00 YGGUSDT     0.45120  7.898176e+06  3.569138e+06   
                          ZECUSDT    20.90000  1.264186e+05  2.647557e+06   
                          ZENUSDT     8.01200  3.044000e+05  2.441487e+06   
                          ZILUSDT     0.02100  1.282211e+08  2.689841e+06   
                          ZRXUSDT     0.31910  8.123198e+06  2.600522e+06   

                                     TrdCnt   TakerBuyVol  TakerBuyQuoVol  \
time                      symbol                                            
2020-06-21 00:00:00+00:00 ADAUSDT   20058.0  6.627875e+07    5.295948e+06   
                          ATOMUSDT   3455.0  9.524376e+04    2.493634e+05   
                          BATUSDT   11214.0  6.305698e+06    1.466296e+06   
                          BCHUSDT   14259.0  1.873830e+04    4.371109e+06   
                          BNBUSDT   17232.0  1.104412e+05    1.785496e+06   
...                                     ...           ...             ...   
2024-02-10 16:00:00+00:00 YGGUSDT   23196.0  4.005292e+06    1.809967e+06   
                          ZECUSDT   19394.0  5.800184e+04    1.214835e+06   
                          ZENUSDT   26285.0  1.398118e+05    1.121572e+06   
                          ZILUSDT   16735.0  6.832354e+07    1.433032e+06   
                          ZRXUSDT   16876.0  3.805058e+06    1.218391e+06   

                                    funding_rate  ...         taOBV  \
time                      symbol                  ...                 
2020-06-21 00:00:00+00:00 ADAUSDT      -0.000036  ...  2.037931e+09   
                          ATOMUSDT      0.000867  ...  1.819203e+05   
                          BATUSDT      -0.000584  ...  8.919682e+07   
                          BCHUSDT       0.000100  ... -1.519552e+05   
                          BNBUSDT       0.000223  ... -2.033896e+06   
...                                          ...  ...           ...   
2024-02-10 16:00:00+00:00 YGGUSDT       0.000100  ...  3.839730e+09   
                          ZECUSDT       0.000100  ...  6.557414e+06   
                          ZENUSDT       0.000100  ...  4.381044e+07   
                          ZILUSDT       0.000100  ...  3.081640e+10   
                          ZRXUSDT       0.000100  ...  4.390101e+09   

                                    taAVGPRICE  taMEDPRICE  taTYPPRICE  \
time                      symbol                                         
2020-06-21 00:00:00+00:00 ADAUSDT     0.0

In [29]:
dffct.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'QuoVol', 'TrdCnt',
       'TakerBuyVol', 'TakerBuyQuoVol', 'funding_rate', 'premIdx_open',
       'premIdx_high', 'premIdx_low', 'premIdx_close', 'spot_Open',
       'spot_High', 'spot_Low', 'spot_Close', 'spot_Vol', 'spot_QuoVol',
       'spot_TrdCnt', 'spot_TakerBuyVol', 'spot_TakerBuyQuoVol', 'oi', 'oiquo',
       'topPosLSCntRt', 'topPosLSRt', 'posLSRt', 'fwdret1', 'bckret1',
       'bckret3', 'bckret5', 'netbuyquo1', 'netbuyquo3', 'netbuyquo5',
       'spot_netbuyquo1', 'spot_netbuyquo3', 'spot_netbuyquo5',
       'spotum_quo_rt1', 'spotum_quo_rt3', 'spotum_quo_rt5', 'avgtrdquo1',
       'avgtrdquo3', 'avgtrdquo5', 'vwap_bias1', 'vwap_bias3', 'vwap_bias5',
       'trd_vwap_bias1', 'trd_vwap_bias3', 'trd_vwap_bias5', 'takerbuy_ratio1',
       'takerbuy_ratio3', 'takerbuy_ratio5', 'taMIDPRICE14', 'taSAR',
       'taSAREXT', 'taADX14', 'taADXR14', 'taAROONDOWN14', 'taAROONUP14',
       'taAROONOSC14', 'taBOP', 'taCCI14', 'taDX14', 'ta

In [15]:
dfcdl = dffct.groupby('symbol',group_keys=False).apply(lambda g: g.ta.cdl_pattern(name="all"))
dfcdl

CDL_2CROWS  CDL_3BLACKCROWS  CDL_3INSIDE  \
time                      symbol                                               
2020-06-21 00:00:00+00:00 ADAUSDT          0.0              0.0          0.0   
                          ATOMUSDT         0.0              0.0          0.0   
                          BATUSDT          0.0              0.0          0.0   
                          BCHUSDT          0.0              0.0          0.0   
                          BNBUSDT          0.0              0.0          0.0   
...                                        ...              ...          ...   
2024-02-10 16:00:00+00:00 YGGUSDT          0.0              0.0          0.0   
                          ZECUSDT          0.0              0.0          0.0   
                          ZENUSDT          0.0              0.0          0.0   
                          ZILUSDT          0.0              0.0          0.0   
                          ZRXUSDT          0.0              0.0          0.0   

                                    CDL_3LINESTRIKE  CDL_3OUTSIDE  \
time                      symbol                                    
2020-06-21 00:00:00+00:00 ADAUSDT               0.0           0.0   
                          ATOMUSDT              0.0           0.0   
                          BATUSDT               0.0           0.0   
                          BCHUSDT               0.0           0.0   
                          BNBUSDT               0.0           0.0   
...                                             ...           ...   
2024-02-10 16:00:00+00:00 YGGUSDT               0.0           0.0   
                          ZECUSDT               0.0           0.0   
                          ZENUSDT               0.0           0.0   
                          ZILUSDT               0.0           0.0   
                          ZRXUSDT               0.0           0.0   

                                    CDL_3STARSINSOUTH  CDL_3WHITESOLDIERS  \
time                      symbol                                            
2020-06-21 00:00:00+00:00 ADAUSDT                 0.0                 0.0   
                          ATOMUSDT                0.0                 0.0   
                          BATUSDT                 0.0                 0.0   
                          BCHUSDT                 0.0                 0.0   
                          BNBUSDT                 0.0                 0.0   
...                                               ...                 ...   
2024-02-10 16:00:00+00:00 YGGUSDT                 0.0                 0.0   
                          ZECUSDT                 0.0                 0.0   
                          ZENUSDT                 0.0                 0.0   
                          ZILUSDT                 0.0                 0.0   
                          ZRXUSDT                 0.0                 0.0   

                                    CDL_ABANDONEDBABY  CDL_ADVANCEBLOCK  \
time                      symbol                                          
2020-06-21 00:00:00+00:00 ADAUSDT                 0.0               0.0   
                          ATOMUSDT                0.0               0.0   
                          BATUSDT                 0.0               0.0   
                          BCHUSDT                 0.0               0.0   
                          BNBUSDT                 0.0               0.0   
...                                               ...               ...   
2024-02-10 16:00:00+00:00 YGGUSDT                 0.0               0.0   
                          ZECUSDT                 0.0               0.0   
                          ZENUSDT                 0.0               0.0   
                          ZILUSDT                 0.0               0.0   
                          ZRXUSDT                 0.0               0.0   

                                    CDL_BELTHOLD  ...  CDL_SPINNINGTOP  \
time                      symbol             

In [28]:
count = dfcdl.query('symbol=="BTCUSDT"').apply(lambda col: col.nunique())
count.shape, count[count>1].shape, count[count>1]

((62,),
 (38,),
 CDL_3INSIDE             3
 CDL_3LINESTRIKE         3
 CDL_3OUTSIDE            3
 CDL_3WHITESOLDIERS      2
 CDL_ADVANCEBLOCK        2
 CDL_BELTHOLD            3
 CDL_CLOSINGMARUBOZU     3
 CDL_DOJI_10_0.1         2
 CDL_DOJISTAR            3
 CDL_DRAGONFLYDOJI       2
 CDL_ENGULFING           3
 CDL_EVENINGDOJISTAR     2
 CDL_EVENINGSTAR         2
 CDL_GAPSIDESIDEWHITE    2
 CDL_GRAVESTONEDOJI      2
 CDL_HAMMER              2
 CDL_HANGINGMAN          2
 CDL_HARAMI              3
 CDL_HARAMICROSS         3
 CDL_HIGHWAVE            3
 CDL_HIKKAKE             5
 CDL_HIKKAKEMOD          5
 CDL_INSIDE              2
 CDL_INVERTEDHAMMER      2
 CDL_LONGLEGGEDDOJI      2
 CDL_LONGLINE            3
 CDL_MARUBOZU            3
 CDL_MATCHINGLOW         2
 CDL_MORNINGDOJISTAR     2
 CDL_MORNINGSTAR         2
 CDL_RICKSHAWMAN         2
 CDL_SEPARATINGLINES     3
 CDL_SHOOTINGSTAR        2
 CDL_SHORTLINE           3
 CDL_SPINNINGTOP         3
 CDL_STALLEDPATTERN      2
 CDL_TAKURI 

In [5]:
datafct = dffct.bckret5.unstack().to_numpy()
display(pd.DataFrame(datafct))

,0,1,2,3,4,5,6,7,8,9,...,258,259,260,261,262,263,264,265,266,267
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.032282,NaN,NaN,NaN,NaN,NaN,-0.006922,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.003044,NaN,NaN,NaN,NaN,NaN,0.001942,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.014079,NaN,NaN,NaN,NaN,NaN,-0.014054,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.032346,NaN,NaN,NaN,NaN,NaN,0.027674,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.022536,NaN,NaN,NaN,NaN,NaN,0.025626,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3985,0.185182,NaN,0.101819,0.007546,0.040861,0.035843,0.040507,0.029879,0.117823,0.037598,...,0.030846,0.009527,-0.001387,NaN,0.018902,0.026340,0.006731,-0.046274,0.032004,0.008130
3986,0.160963,NaN,0.093870,0.032685,0.056312,0.012825,0.041155,0.026000,0.065605,0.050985,...,0.030785,0.021866,0.001382,NaN,0.018994,0.032280,0.028724,-0.041220,0.045387,0.022124
3987,0.239219,NaN,0.077350,0.082434,0.048685,0.026036,0.022415,0.011842,0.036284,0.039582,...,0.021696,0.018416,-0.014364,NaN,0.008707,0.029638,-0.004308,-0.044060,0.021422,0.008775
3988,0.266551,NaN,0.078819,0.124232,0.072849,0.020069,0.038621,0.015193,0.018386,0.035294,...,0.014778,0.015741,0.009094,NaN,0.013580,0.023224,0.002396,-0.041023,0.021579,0.009485


In [16]:
%timeit ta_CMO(datafct, 14)

3.72 ms ± 6.98 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
%timeit pd.DataFrame(datafct).apply(ta.cmo, length=14).to_numpy()

10.8 ms ± 56.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
np.allclose(ta_CMO(datafct, 14), pd.DataFrame(datafct).apply(ta.cmo, length=14).to_numpy(), equal_nan=True)

True

In [25]:
# tszs_60_add(ts_mean_5(netbuyquo3), ta_BBANDSl_5_2_2(spot_Open))

netbuyquo3 = dffct.netbuyquo3.unstack().to_numpy()
spot_Open = dffct.spot_Open.unstack().to_numpy()

x1 = ts_mean(netbuyquo3, 5)
x2 = ta_BBANDSl(spot_Open, 5,5,2,0)
ret = tszs_add(x1, x2, 60)
pd.DataFrame(ret)


,0,1,2,3,4,5,6,7,8,9,...,253,254,255,256,257,258,259,260,261,262
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3910,1.201043,NaN,0.053036,0.107078,0.326916,NaN,NaN,0.265574,0.244867,1.874834,...,2.877297,0.846889,1.188346,NaN,1.351317,2.031437,1.768750,0.657901,-0.171986,1.638280
3911,1.085945,NaN,0.108100,0.076111,0.028208,NaN,NaN,0.077193,0.090805,1.395174,...,3.380551,0.695247,1.060694,NaN,1.116821,1.936355,1.663987,0.666694,-0.366939,1.797745
3912,0.984338,NaN,0.161373,0.091111,-0.128333,NaN,NaN,-0.113998,-0.148184,1.202009,...,3.431515,0.716287,1.109504,NaN,1.018390,1.756186,1.484775,0.682372,-0.396291,2.269082
3913,0.944745,NaN,0.304039,-0.076997,-0.234581,NaN,NaN,-0.098151,-0.377085,0.809893,...,2.988648,0.803640,1.138776,NaN,1.241138,2.169471,1.322816,1.020016,-0.555979,2.309762


In [29]:
import numpy as np

def cs_zscore1(x):
    # 在计算过程中临时忽略特定警告
    with np.errstate(invalid='ignore', divide='ignore'):
        means = np.nanmean(x, axis=1, keepdims=True)
        stds = np.nanstd(x, axis=1, keepdims=True)

        z_scores = np.where(stds > 0, (x - means) / stds, 0)

    # 将原始 NaN 位置的值保持为 NaN
    z_scores[np.isnan(x)] = np.nan

    return z_scores

# 测试数据
x = np.array([
    [1, 2, np.nan, 4],
    [np.nan, np.nan, np.nan, np.nan],
    [7, 8, 9, 10]
])

print(cs_zscore(x))


[[-1.06904497 -0.26726124         nan  1.33630621]
 [        nan         nan         nan         nan]
 [-1.34164079 -0.4472136   0.4472136   1.34164079]]


In [42]:
import numpy as np

def cs_zscore1(x):
    # 检查每行是否全部为 NaN
    all_nan_rows = np.isnan(x).all(axis=1)

    # 为非全 NaN 的行计算均值和标准差
    means = np.nanmean(x[~all_nan_rows], axis=1, keepdims=True)
    stds = np.nanstd(x[~all_nan_rows], axis=1, keepdims=True)

    # 初始化一个全为 NaN 的结果数组
    z_scores = np.full(x.shape, np.nan)

    # 只对非全 NaN 的行进行 Z-score 计算
    z_scores[~all_nan_rows] = (x[~all_nan_rows] - means) / stds

    # 将原始 NaN 位置的值保持为 NaN
    z_scores[np.isnan(x)] = np.nan

    return z_scores

# 测试数据
x = np.array([
    [1, 2, np.nan, 4],
    [np.nan, np.nan, np.nan, np.nan],
    [7, 8, 9, 10]
])

print(cs_zscore1(x))
np.allclose(cs_zscore1(datafct), cs_zscore1(datafct), equal_nan=True)


[[-1.06904497 -0.26726124         nan  1.33630621]
 [        nan         nan         nan         nan]
 [-1.34164079 -0.4472136   0.4472136   1.34164079]]


True

In [44]:
np.nanmean(cs_zscore(datafct), axis=1).max()

8.017185735346263e-16

In [60]:
# 测试数据
x = np.array([
    [1, 2, np.nan, 4],
    # [np.nan, np.nan, np.nan, np.nan],
    [7, 8, 9, 10],
    [17, 18, 19, 110],
    [70, 80, -9, -10]
])
display(x)
dfx = pd.DataFrame(x)
display(dfx)
(dfx - dfx.rolling(2).mean())/dfx.rolling(2).std()

array([[  1.,   2.,  nan,   4.],
       [  7.,   8.,   9.,  10.],
       [ 17.,  18.,  19., 110.],
       [ 70.,  80.,  -9., -10.]])

,0,1,2,3
0,1.0,2.0,NaN,4.0
1,7.0,8.0,9.0,10.0
2,17.0,18.0,19.0,110.0
3,70.0,80.0,-9.0,-10.0


,0,1,2,3
0,NaN,NaN,NaN,NaN
1,0.707107,0.707107,NaN,0.707107
2,0.707107,0.707107,0.707107,0.707107
3,0.707107,0.707107,-0.707107,-0.707107


In [68]:
ts_zscore(x, 4)

array([[        nan,         nan,         nan,         nan],
       [        nan,         nan,         nan,         nan],
       [        nan,         nan,         nan,         nan],
       [ 1.46677648,  1.47449944,         nan, -0.70030874]])

In [74]:
dfx[3][dfx[2].isna()] = np.nan
dfx

,0,1,2,3
0,1.0,2.0,NaN,NaN
1,7.0,8.0,9.0,10.0
2,17.0,18.0,19.0,110.0
3,70.0,80.0,-9.0,-10.0


In [16]:
dff = pd.read_parquet('/home/pennymax/proj/MFM/CrossSection/v1_basic/basic_8h_2/fct_proc.parquet')
dff0 = dff.copy()
display(dff)
mask = dff.Close.isna()
dff.loc[mask, dff.columns != 'Close'] = np.nan
dff.equals(dff0)

Open       High        Low  \
time                      symbol                                      
2020-06-21 00:00:00+00:00 ADAUSDT     0.07941    0.08099    0.07924   
                          ATOMUSDT    2.62500    2.63100    2.59900   
                          BATUSDT     0.23050    0.23530    0.22970   
                          BCHUSDT   232.69000  234.49000  232.34000   
                          BNBUSDT    16.07600   16.33500   16.03500   
...                                       ...        ...        ...   
2024-01-16 16:00:00+00:00 YGGUSDT     0.56250    0.59140    0.55460   
                          ZECUSDT    23.44000   23.50000   23.06000   
                          ZENUSDT     8.74000    8.84900    8.51700   
                          ZILUSDT     0.02281    0.02334    0.02262   
                          ZRXUSDT     0.33880    0.34100    0.33580   

                                        Close        Volume        QuoVol  \
time                      symbol                                            
2020-06-21 00:00:00+00:00 ADAUSDT     0.07988  1.289528e+08  1.030266e+07   
                          ATOMUSDT    2.60000  2.186062e+05  5.718994e+05   
                          BATUSDT     0.23080  1.195930e+07  2.778131e+06   
                          BCHUSDT   233.27000  3.490217e+04  8.138722e+06   
                          BNBUSDT    16.04600  2.257289e+05  3.649726e+06   
...                                       ...           ...           ...   
2024-01-16 16:00:00+00:00 YGGUSDT     0.57590  3.073031e+07  1.765153e+07   
                          ZECUSDT    23.17000  1.609155e+05  3.745056e+06   
                          ZENUSDT     8.55200  6.329710e+05  5.502231e+06   
                          ZILUSDT     0.02295  4.221570e+08  9.680445e+06   
                          ZRXUSDT     0.33810  7.222316e+06  2.443316e+06   

                                      TrdCnt   TakerBuyVol  TakerBuyQuoVol  \
time                      symbol                                             
2020-06-21 00:00:00+00:00 ADAUSDT    20058.0  6.627875e+07    5.295948e+06   
                          ATOMUSDT    3455.0  9.524376e+04    2.493634e+05   
                          BATUSDT    11214.0  6.305698e+06    1.466296e+06   
                          BCHUSDT    14259.0  1.873830e+04    4.371109e+06   
                          BNBUSDT    17232.0  1.104412e+05    1.785496e+06   
...                                      ...           ...             ...   
2024-01-16 16:00:00+00:00 YGGUSDT   127955.0  1.511047e+07    8.677467e+06   
                          ZECUSDT    32139.0  8.289711e+04    1.929701e+06   
                          ZENUSDT    51664.0  2.687420e+05    2.336471e+06   
                          ZILUSDT    46410.0  2.066420e+08    4.740302e+06   
                          ZRXUSDT    23397.0  3.686560e+06    1.247178e+06   

                                    funding_rate  ...  avgtrdquo5  vwap_bias1  \
time                      symbol                  ...                           
2020-06-21 00:00:00+00:00 ADAUSDT      -0.000036  ...  479.795958    1.000185   
                          ATOMUSDT      0.000867  ...  160.740560    1.006199   
                          BATUSDT      -0.000584  ...  244.230070    1.006494   
                          BCHUSDT       0.000100  ...  463.688916    0.999643   
                          BNBUSDT       0.000223  ...  160.587017    1.007642   
...                                          ...  ...         ...         ...   
2024-01-16 16:00:00+00:00 YGGUSDT       0.000100  ...  143.354565    0.997398   
                          ZECUSDT       0.000100  ...  162.796265    1.004464   
                          ZENUSDT       0.000100  ...   90.929182    1.016453   
                          ZILUSDT       0.000100  ...  201.064744    0.999168   
                          ZRXUSDT       0.000100  ...  130.711339    1.000594   

                                    vwap_bias3

True